In [143]:
#auto reload 
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [144]:
from  prep_data import load_data
import torch

In [145]:
(train_data, train_dataloader), (val_data, val_dataloader), (bow_en, bow_fr) = load_data(10000,1000)

c:\Users\Alhus\.conda\envs\projetmla\lib\site-packages\datasets\table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [146]:
for x in train_dataloader:
    print(x["english"]["idx"])
    break

tensor([[   30,    25,     3,  ..., 30000, 30000, 30000],
        [  223, 30000, 30000,  ...,    10,     0, 30000],
        [  465, 30000,    11,  ...,    89,   808,     1],
        ...,
        [  126,     1,     0,  ..., 30000, 30000, 30000],
        [ 1676, 30000,    13,  ..., 30000, 30000, 30000],
        [   13,    85,    73,  ..., 30000, 30000, 30000]], dtype=torch.int16)


In [147]:
def idx_to_word(idx: torch.Tensor, vocab: List):
    return " ".join(list(vocab[idx[(idx < len(vocab))]]))

In [148]:
idx_to_word(torch.tensor([1,2,3,4,5,6,7,8,9,10,30000,30000,30000,30000,30000,30000,30000,30000]), data['bow']['english'])

'of and to a in for is on that by'

In [149]:
def to_vector(ids, vocab_size):
    vecs = torch.zeros(len(ids), vocab_size)
    vecs[range(len(ids)), ids] = 1
    return vecs

class to_vec:
    def __init__(self, tor, vocab_size, lang):
        self.tor = tor
        self.vocab_size = vocab_size
        self.lang = lang

    
    def __call__(self, ids):
        ids = self.tor.tensor(ids["ids_{}".format(self.lang)])
        vecs = self.tor.zeros(len(ids), self.vocab_size)
        vecs[range(len(ids)), ids] = 1
        return vecs

BLEU SCORE

1-EXISTANT NLTK

In [150]:
import nltk 
hypothesis=['Le','chat','se','trouve','sur','le','toit']
reference=['Le','chat','se','trouve','sur','le','toit','bla']
BLEUscore = nltk.translate.bleu_score.sentence_bleu([reference], hypothesis) 
print(BLEUscore)


0.8668778997501817


2-SCORE PERSONNALISE

In [151]:
import collections
import math

def ngram_precision(candidate, reference, n):
    candidate_ngrams = [tuple(candidate[i:i+n]) for i in range(len(candidate)-n+1)]
    reference_ngrams = [tuple(reference[i:i+n]) for i in range(len(reference)-n+1)]

    candidate_ngram_counts = collections.Counter(candidate_ngrams)
    reference_ngram_counts = collections.Counter(reference_ngrams)

    overlap_count = sum((candidate_ngram_counts & reference_ngram_counts).values())
    total_count = max(1, len(candidate_ngrams))

    precision = overlap_count / total_count
    return precision

def bleu_score(candidate, references, weights):
    precisions = []

    for i in range(len(weights)):
        n = i + 1
        precision = ngram_precision(candidate, references, n)
        precisions.append(precision)

    geometric_mean = math.exp(sum(weights[i] * math.log(p) for i, p in enumerate(precisions)))

    brevity_penalty = min(1, len(candidate) / max(len(reference) for reference in references))

    bleu = brevity_penalty * geometric_mean
    return bleu

# Exemple d'utilisation
reference = ['the', 'quick', 'brown', 'fox', 'jumped', 'over', 'the', 'lazy', 'dog']
candidate = ['the', 'fast', 'brown', 'fox', 'jumped', 'over', 'the', 'lazy', 'dog']

candidate=['Le','chat','se','trou','s','le','to','bli']
reference=['Le','chat','se','trouve','sur','le','toit']

weights = [0.25,0.25,0.25] # You can adjust the weights based on your preference

score = bleu_score(candidate, reference, weights)
print("BLEU Score:", score)


BLEU Score: 0.392814650900513


In [152]:
import torch 

#matmul tests: 
batch_size = 2 
Tx = 3 
hidden_size = 4
a = torch.randint(0,10,(batch_size, Tx),dtype=torch.float)
h = torch.randint(0,10,(batch_size, Tx, hidden_size),dtype=torch.float)


In [153]:
a

tensor([[6., 3., 2.],
        [8., 7., 2.]])

In [154]:
h.swapaxes(1,2)

tensor([[[4., 7., 7.],
         [5., 4., 2.],
         [5., 5., 2.],
         [2., 5., 5.]],

        [[9., 8., 8.],
         [2., 8., 8.],
         [7., 1., 0.],
         [6., 9., 4.]]])

In [155]:
h.shape,a.shape

(torch.Size([2, 3, 4]), torch.Size([2, 3]))

In [156]:
(h.swapaxes(1,2) @ a.unsqueeze(2)).squeeze()

tensor([[ 59.,  46.,  49.,  37.],
        [144.,  88.,  63., 119.]])